In [1]:
import os
import re
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# check if the API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("API key is not set")

In [4]:
# initialize the OpenAI client
client = OpenAI()

In [5]:
def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [6]:
def run_openai_agent(query):
    print("### Run OpenAI agent")
    print(f"User query: {query}")
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",  
            messages=[
                {"role": "system", "content": "You are a Python coding assistant. Return a Python function for the given query, followed by a test case using assertion. Format your response with '## Python function' and '## Test case' headers."},
                {"role": "user", "content": query}
            ]
        )
        result = completion.choices[0].message.content
        print("Raw response from OpenAI:")
        print(result)
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

In [7]:
def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [8]:
def extract_code(result):
    retry = False
    print("### Extracting Python function")
    python_function = extract_pattern(result, r'## Python function(.*?)## Test case')
    if not python_function:
        retry = True
        print("Python function failed to generate or wrong output format. Setting retry to True.")
        print("Attempting to extract entire response as Python function.")
        python_function = result
    else:
        # Remove markdown code block delimiters and any leading/trailing whitespace
        python_function = re.sub(r'```python\s*', '', python_function)
        python_function = re.sub(r'\s*```\s*', '', python_function)
        python_function = python_function.strip()

    print("### Extracting test case")
    test_function = extract_pattern(result, r'## Test case(.*?)$')
    if not test_function:
        retry = True
        print("Test function failed to generate or wrong output format. Setting retry to True.")
        print("Attempting to create a simple test case.")
        test_function = "assert True  # Placeholder test"
    else:
        # Remove markdown code block delimiters and any leading/trailing whitespace
        test_function = re.sub(r'```python\s*', '', test_function)
        test_function = re.sub(r'\s*```\s*', '', test_function)
        test_function = test_function.strip()

    print("Extracted Python function:")
    print(python_function)
    print("Extracted test case:")
    print(test_function)

    return python_function, test_function, retry

In [9]:
import traceback

def check_code(python_function, test_function):
    retry = False
    try:
        print("Executing Python function:")
        print(python_function)
        exec(python_function)
        print("Code executed successfully.")
        try:
            print("Executing test case:")
            print(test_function)
            exec(test_function)
            print("Code passed test case.")
        except Exception as e:
            print(f"Test failed with error: {str(e)}")
            print(traceback.format_exc())
            retry = True
            print("Setting retry to True")
    except Exception as e:
        print(f"Code failed with error: {str(e)}")
        print(traceback.format_exc())
        retry = True
        print("Setting retry to True")
    return retry

In [10]:
def run_workflow(query):
    print("### ENTER WORKFLOW")
    i = 0
    max_retries = 3
    retry = True
    while i < max_retries and retry:
        print(f"TRY # {i}")
        i += 1
        result = run_openai_agent(query)
        python_function, test_function, retry = extract_code(result)
        retry = check_code(python_function, test_function)

    if not retry:
        print(f"Validated Python function: ```{python_function}```")
    print("### EXIT WORKFLOW")

In [11]:
run_workflow("How can I remove duplicates from a list")

### ENTER WORKFLOW
TRY # 0
### Run OpenAI agent
User query: How can I remove duplicates from a list
Raw response from OpenAI:
## Python function

```python
def remove_duplicates(input_list):
    """
    Remove duplicates from the list while preserving the order of elements.

    :param input_list: List of elements that may contain duplicates
    :return: List of elements with duplicates removed
    """
    seen = set()
    result = []
    for item in input_list:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
```

## Test case

```python
# Test case for remove_duplicates function
test_list = [1, 2, 2, 3, 4, 4, 5, 1]
assert remove_duplicates(test_list) == [1, 2, 3, 4, 5], f"Test failed! Expected [1, 2, 3, 4, 5] but got {remove_duplicates(test_list)}"
print("Test passed!")
```
### Extracting Python function
### Extracting test case
Extracted Python function:
def remove_duplicates(input_list):
    """
    Remove duplicates from the